In [67]:
from lyricsgenius import Genius
from collections import Counter
import re
from nltk.tokenize import TreebankWordTokenizer
import string
import pandas as pd
import requests
from tqdm.notebook import tqdm

tokenizer = TreebankWordTokenizer()
punct = set(string.punctuation)
punct.update({"''", "``", ""})
genius = Genius('bVEbboB9VeToZE48RaiJwrnAGLz8VbrIdlqnVU70pzJXs_T4Yg6pdPpJrTQDK46p')
genius.verbose = False

In [90]:
path = r'C:\Users\chris\Documents\GitHub\cs4300sp2021-rad338-jsh328-rpp62-cmc447\sample_data'
df = pd.read_csv(path + r"\sample_scraped_data100.csv")

In [100]:
def match(a, b):
    return (a in b) or (b in a)

def get_lyrics(song_name, song_artist):
    s = genius.search_song(song_name, song_artist, get_full_info = False).to_dict()
    query_sn, query_sa = song_name.lower(), song_artist.lower()
    returned_sn, returned_sa = s['artist'].lower(), s['title'].lower()
    
    if match(query_sn, returned_sn) and match(query_sa, returned_sa):
        return s['lyrics']

In [101]:
def get_all_lyrics(df):
    """
    df: DataFrame; uri, name, and artist must be columns in the dataframe
    
    returns a dictionary where key = song uri and value = Counter of tokenized lyrics
    """
    cnt_dict = dict() #dict(uri: Counter(lyric tokens))
    for _, row in tqdm(df.iterrows()):
        uri, name, artist = row[['uri', 'name', 'artist']]
        lyrics = get_lyrics(name, artist)
        if d:
            lyrics = re.sub(r'[\(\[].*?[\)\]]', '', lyrics)
            tokens = [t for t in tokenizer.tokenize(lyrics) if t not in punct]
            cnt = Counter(tokens)
            cnt_dict[uri] = cnt
    return cnt_dict